In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import matplotlib.patches as patches
from dotenv import load_dotenv
from PIL import ImageColor
from matplotlib import pyplot as plt

from utils.s3_bucket import S3Bucket
from utils.predictors import ObjectDetectionPredictor, ImageClassificationPredictor

In [ ]:
load_dotenv("env")

# Object Detection

In [ ]:
od_model_predictor = ObjectDetectionPredictor(
    model_id=os.environ["OBJECT_DETECTION_MODEL_ID"],
    model_version=os.environ["OBJECT_DETECTION_MODEL_VERSION"],
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
)

In [ ]:
%%time
od_model_predictor.deploy(
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
    instance_count=int(os.environ["INFERENCE_INSTANCE_COUNT"]),
)

In [ ]:
bucket_name = "ava-cv-raw-photo-bucket"
key = os.path.join("10-plants", "GoogleImages", "basil", "basilplant0.jpeg")

bucket = S3Bucket(
    bucket_name=bucket_name,
    region_name=os.environ["REGION_NAME"]
)
image = bucket[key]
image_np = np.array(image)
plt.figure(figsize=(10, 10))
plt.imshow(image_np)
plt.show()

In [ ]:
%%time
normalized_boxes, classes_names, confidences, labels = od_model_predictor.predict(image)

In [ ]:
def display_predictions(image_np, normalized_boxes, classes_names, confidences):
    n_boxes = len(normalized_boxes)
    print(f"Number of bounding boxes: {n_boxes}")
    colors = list(ImageColor.colormap.values())
    plt.figure(figsize=(10, 10))
    ax = plt.axes()
    ax.imshow(image_np)

    for idx in range(n_boxes):
        left, bot, right, top = normalized_boxes[idx]
        x, w = [val * image_np.shape[1] for val in [left, right - left]]
        y, h = [val * image_np.shape[0] for val in [bot, top - bot]]
        color = colors[hash(classes_names[idx]) % len(colors)]
        rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor="none")
        ax.add_patch(rect)
        ax.text(
            x,
            y,
            "{} {:.0f}%".format(classes_names[idx], confidences[idx] * 100),
            bbox=dict(facecolor="white", alpha=0.5),
        )

In [ ]:
display_predictions(image_np, normalized_boxes, classes_names, confidences)

In [ ]:
# Keep bounding boxes that are part of classes_to_keep
classes_to_keep = ["potted plant"]
n_boxes = len(normalized_boxes)
normalized_boxes = [normalized_boxes[i] for i in range(n_boxes) if classes_names[i] in classes_to_keep]
confidences = [confidences[i] for i in range(n_boxes) if classes_names[i] in classes_to_keep]
classes_names = [classes_names[i] for i in range(n_boxes) if classes_names[i] in classes_to_keep]
display_predictions(image_np, normalized_boxes, classes_names, confidences)

In [ ]:
# Keep bounding boxes with confidence higher than min_confidence
min_confidence = float(os.environ["OBJECT_DETECTION_MIN_CONFIDENCE"])
n_boxes = len(normalized_boxes)
normalized_boxes = [normalized_boxes[i] for i in range(n_boxes) if confidences[i] >= min_confidence]
classes_names = [classes_names[i] for i in range(n_boxes) if confidences[i] >= min_confidence]
confidences = [confidences[i] for i in range(n_boxes) if confidences[i] >= min_confidence]
display_predictions(image_np, normalized_boxes, classes_names, confidences)

In [ ]:
cropped_images = []
for normalized_box in normalized_boxes:
    left, top, right, bot = normalized_box
    left, right = [val * image_np.shape[1] for val in [left, right]]
    bot, top = [val * image_np.shape[0] for val in [bot, top]]
    cropped_image = image.crop((left, top, right, bot))
    cropped_images.append(cropped_image)

In [ ]:
od_model_predictor.delete()

# Image Classification

In [ ]:
ic_model_predictor = ImageClassificationPredictor(
    model_id=os.environ["IMAGE_CLASSIFICATION_MODEL_ID"],
    model_version=os.environ["IMAGE_CLASSIFICATION_MODEL_VERSION"],
    model_uri=os.environ["IMAGE_CLASSIFICATION_MODEL_URI"],
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
)

In [ ]:
%%time
ic_model_predictor.deploy(
    instance_type=os.environ["INFERENCE_INSTANCE_TYPE"],
    instance_count=int(os.environ["INFERENCE_INSTANCE_COUNT"]),
)

In [ ]:
for cropped_image in cropped_images:
    labels, probabilities = ic_model_predictor.predict(cropped_image)
    plt.figure(figsize=(5, 5))
    plt.imshow(np.array(cropped_image))
    plt.show()
    for label, probability in zip(labels, probabilities):
        print(f"{label}: {probability*100:.2f}%")
    print("")

In [ ]:
ic_model_predictor.delete()